In [1]:
import tensorflow as tf

In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape = (None, 5))

In [4]:
# To use a layer, simply call it, now available in tf2 eager execution
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [5]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.08421665, -0.10728902,  0.01252663,  0.41816443, -0.32710463,
         -0.10855794, -0.4753332 , -0.08511233,  0.3452978 ,  0.16453886],
        [-0.30516437,  0.59640235,  0.39871424,  0.3727972 ,  0.00167149,
          0.09404492,  0.18964452, -0.06258428,  0.45141834,  0.53857213],
        [ 0.35323054, -0.03484994,  0.35637778,  0.23185897,  0.31196254,
         -0.28198263, -0.06325471,  0.30035633, -0.0759092 , -0.24335349],
        [-0.18869317,  0.12801409,  0.14818776,  0.03936666,  0.5547325 ,
          0.15526259,  0.2841044 ,  0.3667307 ,  0.29313684,  0.41966683],
        [ 0.37449378, -0.46483845, -0.29336753,  0.20945579, -0.35888886,
         -0.39375252, -0.15088776, -0.23561558,  0.32949573, -0.54438996]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [6]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.08421665, -0.10728902,  0.01252663,  0.41816443, -0.32710463,
         -0.10855794, -0.4753332 , -0.08511233,  0.3452978 ,  0.16453886],
        [-0.30516437,  0.59640235,  0.39871424,  0.3727972 ,  0.00167149,
          0.09404492,  0.18964452, -0.06258428,  0.45141834,  0.53857213],
        [ 0.35323054, -0.03484994,  0.35637778,  0.23185897,  0.31196254,
         -0.28198263, -0.06325471,  0.30035633, -0.0759092 , -0.24335349],
        [-0.18869317,  0.12801409,  0.14818776,  0.03936666,  0.5547325 ,
          0.15526259,  0.2841044 ,  0.3667307 ,  0.29313684,  0.41966683],
        [ 0.37449378, -0.46483845, -0.29336753,  0.20945579, -0.35888886,
         -0.39375252, -0.15088776, -0.23561558,  0.32949573, -0.54438996]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

In [7]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape = [int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)

In [8]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

In [9]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']
